In [1]:
%load_ext autoreload
%autoreload 2

import math
import numpy as np
from tqdm import tqdm
import _dfs_codes as dfs

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optimizers
from torch.utils.data import Dataset, DataLoader

from ogb.graphproppred import Evaluator
from ogb.graphproppred import GraphPropPredDataset
from ogb.graphproppred.mol_encoder import AtomEncoder, BondEncoder

import torch_geometric as tg
from torch_geometric.data import Data
import dfs_code
import networkx as nx

from focal_loss.focal_loss import FocalLoss

In [2]:
import sys
sys.path = ['/home/chrisw/Documents/projects/2021/graph-transformer/src'] + sys.path
from dfs_transformer import EarlyStopping, DFSCodeClassifier, DFSCodeSeq2SeqFC
import wandb
import random 
import os
#torch.multiprocessing.set_sharing_strategy('file_system')
torch.multiprocessing.set_sharing_strategy('file_descriptor')
manualSeed = 44
random.seed(manualSeed)
torch.manual_seed(manualSeed)
np.random.seed(manualSeed)
print("Random Seed: ", manualSeed)

Random Seed:  44


In [3]:
wandb.init(project='molhiv-transformer', entity='chrisxx')

config = wandb.config
config.max_nodes = 100
config.max_edges = 200
config.nlayers = 6
config.emb_dim = 50
config.nhead = 5
config.dim_feedforward = 2*(5*config.emb_dim)
config.lr = 0.0003
config.n_epochs = 25
config.patience = 5
config.factor = 0.95
config.minimal_lr = 6e-8
config.batch_size = 256
config.valid_patience = 100
config.valid_minimal_improvement=0.00
config.pretrained_model_dir = '../models/chembl/transformer/mini/'
config.model_dir = '../models/chembl/transformer/mini/molhiv/'
config.num_workers = 4

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chrisxx (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
model = DFSCodeSeq2SeqFC(n_atoms=118, n_bonds=4, emb_dim=50, nhead=5, nlayers=6, max_nodes=100, max_edges=400,
                         atom_encoder=nn.Embedding(118, 50), bond_encoder=nn.Linear(4, 50))

In [5]:
model.load_state_dict(torch.load(config.pretrained_model_dir+'checkpoint.pt'))

<All keys matched successfully>

In [6]:
os.makedirs(config.model_dir, exist_ok=True)

In [7]:
class SplittedDataset(Dataset):
    def __init__(self, D, S):
        super().__init__()
        self.D, self.S = D, S

    def __len__(self):
        return len(self.S)

    def __getitem__(self, idx):
        return self.D[self.S[idx]]


class MolhivDFSCodeDataset(Dataset):
    def __init__(self, max_nodes=40):
        super().__init__()
        self.maxN = max_nodes
        self.molhiv = GraphPropPredDataset(name='ogbg-molhiv')
        self.throw_labels = lambda code: list(map(lambda c: c[:2] + c[-3:], code))

    def __len__(self):
        return len(self.molhiv)

    def __getitem__(self, idx):
        added_e, Edges = [], []
        G, L = self.molhiv[idx]
        
        edge_index = []
        elabels = []
        efeats = []
        for edge, feat in zip(G['edge_index'].T, G['edge_feat']):
            if edge[0] < self.maxN and edge[1] < self.maxN:
                edge_index.append(edge.tolist())
                elabels.append(feat[0])
                efeats.append(feat.tolist())
        edge_index = np.asarray(edge_index).astype(np.int64).T
        efeats = np.asarray(efeats)
        vlabels = G['node_feat'][:self.maxN, 0].tolist()
        
        # only keep largest connected component
        edges_coo = edge_index.copy().T
        g = nx.Graph()
        g.add_nodes_from(np.arange(len(vlabels)))
        g.add_edges_from(edges_coo.tolist())

        ccs = list(nx.connected_components(g))
        largest_cc = ccs[np.argmax([len(cc) for cc in ccs])]
        node_ids = np.asarray(list(largest_cc))

        x = G['node_feat'][:min(G['num_nodes'], self.maxN)][node_ids]
        z = x[:, 0]
        edges_cc = []
        edge_feats = []
        edge_labels = []
        old2new = {old:new for new, old in enumerate(node_ids)}
        for idx, (u, v) in enumerate(edges_coo):
            if u in node_ids and v in node_ids:
                edges_cc += [[old2new[u], old2new[v]]]
                edge_feats += [efeats[idx].tolist()]
                edge_labels += [elabels[idx]]
        edge_index = torch.tensor(edges_cc, dtype=torch.long).T
        edge_attr = torch.tensor(edge_feats, dtype=torch.int32)


        data = Data(x=x, z=z, pos=None, edge_index=edge_index,
                    edge_attr=edge_attr, y=None)
        
        
        if config.max_nodes < 50:
            Edges, _ = dfs_code.min_dfs_code_from_torch_geometric(data, z.tolist(), edge_labels)
        else:
            Edges, _ = dfs_code.rnd_dfs_code_from_torch_geometric(data, z.tolist(), edge_labels)
        #TODO: don't forget that we use random dfs codes here.
        edge_attr_new = []
        emap = {0:0, 1:1, 2:3, 3:2, 4:0} #ogb bond types: 'SINGLE','DOUBLE','TRIPLE','AROMATIC','misc'
        for etype in edge_attr[:,0].cpu().numpy():
            etype_np = np.zeros(4, dtype=np.int32)
            etype_np[emap[etype]] = 1
            edge_attr_new += [etype_np.tolist()]
        edge_attr = torch.tensor(edge_attr_new, dtype=torch.float)
        return torch.LongTensor(Edges), torch.tensor(z, dtype=torch.long), edge_attr, torch.tensor(L[0]) 
    #C (code) #N (node features) #E (edge features) and label




In [8]:
D = MolhivDFSCodeDataset(max_nodes=config.max_nodes)
data_split = D.molhiv.get_idx_split()
val_sampler = torch.utils.data.SubsetRandomSampler(data_split['valid'])
test_sampler = torch.utils.data.SubsetRandomSampler(data_split['test'])
train_sampler = torch.utils.data.SubsetRandomSampler(data_split['train'])
#valdata = [D[i] for i in data_split['valid']]
#testdata = [D[i] for i in data_split['test']]
#traindata = [D[i] for i in data_split['train']]

In [9]:
D[1123]

(tensor([[ 0,  1,  5,  1,  5,  3,  5,  2],
         [ 1,  2,  5,  0,  5,  2,  3,  1],
         [ 2,  3,  5,  2,  6,  1,  1,  0],
         [ 1,  4,  5,  0,  5,  2, 18, 10],
         [ 4,  5,  5,  3,  5, 10, 20, 11],
         [ 5,  6,  5,  3,  5, 11, 22, 12],
         [ 6,  7,  5,  3,  5, 12, 24, 13],
         [ 7,  8,  5,  0, 16, 13, 26, 14],
         [ 7,  9,  5,  3,  5, 13, 28, 15],
         [ 9, 10,  5,  3,  5, 15, 30, 16],
         [10,  4,  5,  3,  5, 16, 34, 10],
         [ 0, 11,  5,  0,  5,  3,  6,  4],
         [11, 12,  5,  3,  5,  4,  8,  5],
         [12, 13,  5,  3,  5,  5, 10,  6],
         [13, 14,  5,  3,  5,  6, 12,  7],
         [14, 15,  5,  3,  5,  7, 14,  8],
         [15, 16,  5,  3,  5,  8, 16,  9],
         [16, 11,  5,  3,  5,  9, 32,  4]]),
 tensor([ 6,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5, 16,  5,  5]),
 tensor([[0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [

In [10]:
val_loader = DataLoader(D, sampler=val_sampler, batch_size=config.batch_size, pin_memory=True, collate_fn=lambda x:x, num_workers=config.num_workers)
test_loader = DataLoader(D, sampler=test_sampler, batch_size=config.batch_size, pin_memory=True, collate_fn=lambda x:x, num_workers=config.num_workers)
train_loader = DataLoader(D, sampler=train_sampler, batch_size=config.batch_size, pin_memory=True, collate_fn=lambda x:x, num_workers=config.num_workers)

In [11]:
loss_h, val_h = [], []
evaluator = Evaluator(name='ogbg-molhiv')
to_cuda = lambda T: map(lambda t: t.cuda(), T)
to_cuda = lambda T: [t.cuda() for t in T]

model_head = nn.Linear(2*(config.emb_dim*5), 1)
#M = nn.Sequential(model, model_head)
#
optim = optimizers.Adam(list(model_head.parameters())+list(model.parameters()), lr=config.lr)#, betas=(0.9, 0.98))#, eps=1e-7)

lr_scheduler = optimizers.lr_scheduler.ReduceLROnPlateau(optim, mode='min', verbose=True, patience=config.patience, factor=config.factor)
early_stopping_head = EarlyStopping(patience=config.valid_patience, delta=config.valid_minimal_improvement,
                              path=config.model_dir+'checkpoint_head.pt')
early_stopping_feats = EarlyStopping(patience=config.valid_patience, delta=config.valid_minimal_improvement,
                              path=config.model_dir+'checkpoint_feats.pt')

In [12]:
model.cuda()
model_head.cuda()
criterion = torch.nn.BCEWithLogitsLoss()

In [13]:
def score(loader, model, model_head):
    val_roc = 0
    with torch.no_grad():
        full_preds, target = [], []
        for batch in tqdm(loader):
            C, N, E, y = zip(*batch)
            y = torch.Tensor(y).cuda()
            features = model.encode(to_cuda(C), to_cuda(N), to_cuda(E))
            pred = model_head(features).squeeze()
            target.extend(y.cpu().tolist())
            full_preds.extend((1. * (0.5 < torch.sigmoid(pred))).cpu().tolist())

        val_roc = evaluator.eval({'y_true': np.expand_dims(target, axis=1),
                                  'y_pred': np.expand_dims(full_preds, axis=1)})['rocauc']
    return val_roc

In [14]:
try:
    for epoch in range(config.n_epochs):  
        model.train()
        tot_loss = 0
        pbar = tqdm(enumerate(train_loader))
        for i, batch in pbar:
            C, N, E, y = zip(*batch)
            y = torch.Tensor(y).cuda()
            features = model.encode(to_cuda(C), to_cuda(N), to_cuda(E))
            pred = model_head(features).squeeze()
            optim.zero_grad()
            loss = criterion(pred, y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
            torch.nn.utils.clip_grad_norm_(model_head.parameters(), 0.5)
            optim.step()
            tot_loss = (tot_loss*i + loss.item())/(i+1)
            pbar.set_description('Epoch %d: BCE %2.6f'%(epoch+1, tot_loss))
        model.eval()
        val_roc = score(val_loader, model, model_head)

        epoch += 1
        val_h.append(val_roc)
        loss_h.append(tot_loss)

        lr_scheduler.step(tot_loss)
        early_stopping_head(-val_roc, model_head)
        early_stopping_feats(-val_roc, model)
        curr_lr = list(optim.param_groups)[0]['lr']


        wandb.log({'BCE':loss_h[-1], 
                   'ROCAUC valid':val_roc,
                   'learning rate':curr_lr})


        if early_stopping_head.early_stop:
            break

        if curr_lr < config.minimal_lr:
            break

        print(f'\nepoch: {len(loss_h)} - loss: {loss_h[-1]} - val: {val_h[-1]}', flush=True)
except KeyboardInterrupt:
    print('keyboard interrupt caught')


Epoch 1: BCE 0.159298: : 129it [01:11,  1.80it/s]
  0%|                                                                                                                                                                                       | 0/17 [00:00<?, ?it/s][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:06<00:00,  2.59it/s]


epoch: 1 - loss: 0.1592979107716287 - val: 0.5604883156966491



[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
Epoch 2: BCE 0.143678: : 129it [01:11,  1.79it/s]
  0%|                                                                                                                                                                                       | 0/17 [00:00<?, ?it/s][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-po


epoch: 2 - loss: 0.14367760481067413 - val: 0.5964092813051146


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
Epoch 3: BCE 0.137780: : 129it [01:11,  1.80it/s]
  0%|                                                                                                                                                                                       | 0/17 [00:00<?, ?it/s][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-poo

...


epoch: 23 - loss: 0.11501003839364347 - val: 0.6407352292768959


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
Epoch 24: BCE 0.117208: : 129it [01:11,  1.80it/s]
  0%|                                                                                                                                                                                       | 0/17 [00:00<?, ?it/s][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-po

Epoch    24: reducing learning rate of group 0 to 2.8500e-04.
EarlyStopping counter: 1 out of 100
EarlyStopping counter: 1 out of 100

epoch: 24 - loss: 0.11720843408112377 - val: 0.6217206790123456



[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
Epoch 25: BCE 0.120025: : 129it [01:11,  1.81it/s]
  0%|                                                                                                                                                                                       | 0/17 [00:00<?, ?it/s][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-p

EarlyStopping counter: 2 out of 100
EarlyStopping counter: 2 out of 100

epoch: 25 - loss: 0.12002549600578094 - val: 0.6035741843033509


In [15]:
model_head.load_state_dict(torch.load(config.model_dir+'checkpoint_head.pt'))
model.load_state_dict(torch.load(config.model_dir+'checkpoint_feats.pt'))
model.eval()

test_roc = score(test_loader, model, model_head)

wandb.log({'ROCAUC test': test_roc})
print(f'\ntest AUROC: {test_roc} at best val epoch {np.argmax(val_h)+1}')

  0%|                                                                                                                                                                                       | 0/17 [00:00<?, ?it/s][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:06<00:00,  2.55it/s]


test AUROC: 0.6128739450356323 at best val epoch 23
